# VAE

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.mixture import BayesianGaussianMixture
from scipy.stats import multivariate_normal
import joblib
from itertools import product
import numpy as np
from scipy.spatial.distance import cdist
import optuna
from functools import partial
from sklearn.model_selection import ShuffleSplit

## 1. Vanilla TVAE
Using plain TVAE generate synthetic data as baseline model.

In [14]:
# Read the file and select numerical column
train_real = pd.read_csv("data/merged_data/train_set.csv").select_dtypes(include="number")
test_real  = pd.read_csv("data/merged_data/test_set.csv").select_dtypes(include="number")
df_copu  = pd.read_csv ("data/Copula_data/synthetic_clayton_split.csv").select_dtypes(include="number")

In [15]:
# Train / Test data split
train_copu, test_copu = train_test_split(df_copu, test_size=0.2,
                                            shuffle=True, random_state=42)

In [16]:
# Generate metadata
meta = SingleTableMetadata()
for c in train_real.columns:
    meta.add_column(c, sdtype="numerical")

In [17]:
# Define TVAE model and training
def train_tvae(df, meta, epochs=300, batch=256):
    tvae = TVAESynthesizer(metadata=meta, epochs=epochs,
                           batch_size=batch, cuda=True)
    tvae.fit(df)
    return tvae

# 1. Train with only real data
tvae_real = train_tvae(train_real, meta)
fake_real = tvae_real.sample(10_000)

# 2. Train with pure copula 
tvae_copu = train_tvae(train_copu, meta)
fake_copu = tvae_copu.sample(10_000)

/Users/xz2821/anaconda3/envs/IRP/lib/python3.8/site-packages/sdv/single_table/base.py:163: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

/Users/xz2821/anaconda3/envs/IRP/lib/python3.8/site-packages/sdv/single_table/base.py:129: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.

/Users/xz2821/anaconda3/envs/IRP/lib/python3.8/site-packages/sdv/single_table/base.py:163: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.



In [18]:
# 3. Train with mixing data (mixing ratio = [1:1, 1:2, 1:4, 1:8])
def build_mixes(train_real: pd.DataFrame,
                train_copu: pd.DataFrame,
                ratios=(1, 2, 4, 8),
                random_state: int = 42):
    n_real = len(train_real)
    mixes = {}

    for r in ratios:
        n_fake = n_real * r
        fake_subset = train_copu.sample(
            n_fake, replace=False, random_state=random_state
        )
        df_mix = pd.concat(
            [train_real, fake_subset], ignore_index=True
        ).sample(frac=1, random_state=random_state)  
        mixes[r] = df_mix.reset_index(drop=True)

    return mixes

mix_dict = build_mixes(train_real, train_copu)  # Default: 1,2,4,8
df_1x = mix_dict[1]   # 1 : 1
df_2x = mix_dict[2]   # 1 : 2
df_4x = mix_dict[4]   # 1 : 4
df_8x = mix_dict[8]   # 1 : 8

tvae_1x = train_tvae(df_1x, meta)
fake_1x = tvae_1x.sample(10_000)

tvae_2x = train_tvae(df_2x, meta)
fake_2x = tvae_2x.sample(10_000)

tvae_4x = train_tvae(df_4x, meta)
fake_4x = tvae_4x.sample(10_000)

tvae_8x = train_tvae(df_8x, meta)
fake_8x = tvae_8x.sample(10_000)

/Users/xz2821/anaconda3/envs/IRP/lib/python3.8/site-packages/sdv/single_table/base.py:163: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

/Users/xz2821/anaconda3/envs/IRP/lib/python3.8/site-packages/sdv/single_table/base.py:163: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

/Users/xz2821/anaconda3/envs/IRP/lib/python3.8/site-packages/sdv/single_table/base.py:163: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

/Users/xz2821/anaconda3/envs/IRP/lib/python3.8/site-packages/sdv/single_table/base.py:163: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.



In [19]:
# Test the results
TARGET = "fragmentation_index"          # Our prediction target

def xgb_score(df_train, df_test):
    X_tr, y_tr = df_train.drop(columns=[TARGET]), df_train[TARGET]
    X_te, y_te = df_test.drop(columns=[TARGET]), df_test[TARGET]
    mdl = XGBRegressor(
        n_estimators=400, max_depth=4, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, random_state=0
    )
    mdl.fit(X_tr, y_tr)
    pred = mdl.predict(X_te)
    return r2_score(y_te, pred), mean_squared_error(y_te, pred, squared=False)

In [20]:
# TSTR
r2_tstr_baseline, rmse_tstr_baseline = xgb_score(train_real, test_real)   # always-real baseline
r2_tstr_real,     rmse_tstr_real     = xgb_score(fake_real,  test_real)   # TVAE-real
r2_tstr_copu,     rmse_tstr_copu     = xgb_score(fake_copu,  test_real)   # Copula + real

print(f"TSTR Baseline : R^2={r2_tstr_baseline:.3f}  RMSE={rmse_tstr_baseline:.3f}")
print(f"TSTR RealOnly : R^2={r2_tstr_real    :.3f}  RMSE={rmse_tstr_real    :.3f}")
print(f"TSTR CopuOnly : R^2={r2_tstr_copu    :.3f}  RMSE={rmse_tstr_copu    :.3f}")

# TRTS
r2_trts_baseline, rmse_trts_baseline = xgb_score(train_real, test_copu)   # real to copula split
r2_trts_real,     rmse_trts_real     = xgb_score(train_real, fake_real)   # real to TVAE-real
r2_trts_copu,     rmse_trts_copu     = xgb_score(train_copu, fake_copu)   # copula to copula

print(f"TRTS Baseline : R^2 ={r2_trts_baseline:.3f}  RMSE={rmse_trts_baseline:.3f}")
print(f"TRTS RealOnly : R^2 ={r2_trts_real    :.3f}  RMSE={rmse_trts_real    :.3f}")
print(f"TRTS CopuOnly : R^2 ={r2_trts_copu    :.3f}  RMSE={rmse_trts_copu    :.3f}")

TSTR Baseline : R^2=0.983  RMSE=0.425
TSTR RealOnly : R^2=0.434  RMSE=2.489
TSTR CopuOnly : R^2=-0.613  RMSE=4.203
TRTS Baseline : R^2 =-0.388  RMSE=3.496
TRTS RealOnly : R^2 =-0.012  RMSE=2.702
TRTS CopuOnly : R^2 =-0.050  RMSE=1.749


In [21]:
# Mix data test:
# TSTR
r2_tstr_1x, rmse_tstr_1x = xgb_score(fake_1x, test_real)   
r2_tstr_2x, rmse_tstr_2x = xgb_score(fake_2x, test_real)   
r2_tstr_4x, rmse_tstr_4x = xgb_score(fake_4x, test_real)  
r2_tstr_8x, rmse_tstr_8x = xgb_score(fake_8x, test_real)  

print(f"TSTR 1x : R^2={r2_tstr_1x:.3f}  RMSE={rmse_tstr_1x:.3f}")
print(f"TSTR 2x : R^2={r2_tstr_2x:.3f}  RMSE={rmse_tstr_2x:.3f}")
print(f"TSTR 4x : R^2={r2_tstr_4x:.3f}  RMSE={rmse_tstr_4x:.3f}")
print(f"TSTR 8x : R^2={r2_tstr_8x:.3f}  RMSE={rmse_tstr_8x:.3f}")

TSTR 1x : R^2=0.289  RMSE=2.791
TSTR 2x : R^2=0.385  RMSE=2.596
TSTR 4x : R^2=0.253  RMSE=2.859
TSTR 8x : R^2=-0.359  RMSE=3.857


## 2. TAVE + Annealing
Basing on the logic of paper “Generating Sentences from a Continuous Space”, we can apply annealing system in TVAE.

In [22]:
# Data preprocessing
class NumScaler:
    """Simple packaging for fit / transform / inverse_transform"""
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, df: pd.DataFrame):
        self.columns = df.columns
        self.scaler.fit(df.values)

    def transform(self, df: pd.DataFrame) -> torch.Tensor:
        return torch.tensor(self.scaler.transform(df[self.columns].values),
                            dtype=torch.float32)

    def inverse_transform(self, tensor: torch.Tensor) -> pd.DataFrame:
        arr = self.scaler.inverse_transform(tensor.detach().cpu().numpy())
        return pd.DataFrame(arr, columns=self.columns)


# Define VAE structure
class VAE(nn.Module):
    def __init__(self, input_dim: int, latent_dim=32,
                 h=[128, 64]):
        super().__init__()
        h1, h2 = h
        # Encoder
        self.enc_fc1 = nn.Linear(input_dim, h[0])
        self.enc_fc2 = nn.Linear(h[0], h[1])
        self.mu      = nn.Linear(h[1], latent_dim)
        self.logvar  = nn.Linear(h[1], latent_dim)
        # Decoder
        self.dec_fc1 = nn.Linear(latent_dim, h[1])
        self.dec_fc2 = nn.Linear(h[1], h[0])
        self.recon   = nn.Linear(h[0], input_dim)

    def encode(self, x):
        h = F.relu(self.enc_fc1(x))
        h = F.relu(self.enc_fc2(h))
        return self.mu(h), self.logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.dec_fc1(z))
        h = F.relu(self.dec_fc2(h))
        return self.recon(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z          = self.reparameterize(mu, logvar)
        recon_x    = self.decode(z)
        return recon_x, mu, logvar

# 3)  KL‑Annealing
class TVAEAnneal:
    def __init__(self,
                 metadata,
                 latent_dim=32,
                 hidden_sizes=(128, 64),
                 beta_max=1.0,
                 batch_size=256,
                 device=None):

        self.metadata = metadata
        self.latent   = latent_dim
        self.h_sizes  = hidden_sizes
        self.beta_max = beta_max
        self.batch    = batch_size
        self.device   = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.scaler   = NumScaler()

    # fit
    def fit(self, df: pd.DataFrame,
            epochs=300, warmup_epochs=30, lr=2e-3,
            sched='linear'):

        # 1) scale
        self.scaler.fit(df)
        tensor_data = self.scaler.transform(df)

        loader = DataLoader(
            TensorDataset(tensor_data),
            batch_size=min(self.batch, len(tensor_data)),  # Protect batch size > row number
            shuffle=True, drop_last=False)

        # 2) model / opt
        dim = tensor_data.shape[1]
        self.model = VAE(dim,
                         latent_dim=self.latent,
                         h=list(self.h_sizes)).to(self.device)

        opt = torch.optim.Adam(self.model.parameters(), lr=lr)

        total_steps = epochs * len(loader)
        warm_steps  = warmup_epochs * len(loader)       # Unify Variable name

        # Beta Annealing
        def beta(step: int) -> float:
            if sched == 'linear':
                core = min(1.0, step / warm_steps)
            elif sched == 'sigmoid':
                k = 6 / warm_steps
                core = 1 / (1 + np.exp(-k * (step - warm_steps/2)))
            elif sched == 'cyclic':
                cycles = 4
                cycle_len = total_steps // cycles
                pos = step % cycle_len
                core = min(1.0, pos / warm_steps)
            else:                     
                core = 1.0
            return core * self.beta_max                # Add beta_max

        # 3) train loop
        global_step = 0
        self.model.train()
        for epoch in range(epochs):
            for (x_batch,) in loader:
                global_step += 1

                x_batch = x_batch.to(self.device)
                recon, mu, logvar = self.model(x_batch)

                mse = F.mse_loss(recon, x_batch, reduction='mean')
                kl  = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
                loss = mse + beta(global_step) * kl

                opt.zero_grad()
                loss.backward()
                opt.step()

            if (epoch + 1) % 50 == 0:
                print(f"Epoch {epoch+1}/{epochs} | "
                      f"Loss={loss.item():.4f} | "
                      f"MSE={mse.item():.4f} | KL={kl.item():.4f} | "
                      f"β={beta(global_step):.3f}")

    # sample
    def sample(self, n: int) -> pd.DataFrame:
        self.model.eval()
        with torch.no_grad():
            z = torch.randn(n, self.latent).to(self.device)
            gen = self.model.decode(z)
        return self.scaler.inverse_transform(gen)


# Train & generate
def train_tvae_anneal(df_train, metadata,
                      epochs=300, warmup=30):
    synthesizer = TVAEAnneal(metadata, batch_size=256)
    synthesizer.fit(df_train, epochs=epochs, warmup_epochs=warmup)
    return synthesizer


In [23]:
# Train with real data only
tvae_real = TVAEAnneal(
    metadata=meta,
    batch_size=256,      
    device="cpu")         # CPU is enought for the data size

tvae_real.fit(
    train_real,
    epochs=300,           # Train epochs from fit
    warmup_epochs=30)     # KL-annealing

fake_real = tvae_real.sample(10_000)

# Train with Copula data only
tvae_copu = TVAEAnneal(
    metadata=meta,
    batch_size=256,      
    device="cpu")         

tvae_copu.fit(
    train_copu,
    epochs=300,           
    warmup_epochs=30)     

fake_copu = tvae_copu.sample(10_000)


Epoch 50/300 | Loss=0.4476 | MSE=0.2705 | KL=0.1771 | β=1.000
Epoch 100/300 | Loss=0.3812 | MSE=0.2009 | KL=0.1804 | β=1.000
Epoch 150/300 | Loss=0.3507 | MSE=0.1678 | KL=0.1829 | β=1.000
Epoch 200/300 | Loss=0.3271 | MSE=0.1512 | KL=0.1759 | β=1.000
Epoch 250/300 | Loss=0.2989 | MSE=0.1218 | KL=0.1772 | β=1.000
Epoch 300/300 | Loss=0.2835 | MSE=0.1198 | KL=0.1637 | β=1.000
Epoch 50/300 | Loss=0.3530 | MSE=0.1217 | KL=0.2313 | β=1.000
Epoch 100/300 | Loss=0.3584 | MSE=0.1283 | KL=0.2300 | β=1.000
Epoch 150/300 | Loss=0.3467 | MSE=0.1136 | KL=0.2332 | β=1.000
Epoch 200/300 | Loss=0.3422 | MSE=0.1103 | KL=0.2320 | β=1.000
Epoch 250/300 | Loss=0.3446 | MSE=0.1159 | KL=0.2287 | β=1.000
Epoch 300/300 | Loss=0.3481 | MSE=0.1154 | KL=0.2327 | β=1.000


In [24]:
# TSTR
r2_tstr_real, rmse_tstr_real = xgb_score(fake_real, test_real)
r2_tstr_copu, rmse_tstr_copu = xgb_score(fake_copu, test_real)

# TRTS
r2_trts_real, rmse_trts_real = xgb_score(train_real, fake_real)
r2_trts_copu, rmse_trts_copu = xgb_score(train_copu, fake_copu)

# Print
print(f"TSTR RealOnly : R^2={r2_tstr_real:.3f}  RMSE={rmse_tstr_real:.3f}")
print(f"TSTR CopuOnly : R^2={r2_tstr_copu:.3f}  RMSE={rmse_tstr_copu:.3f}")

print(f"TRTS RealOnly : R^2={r2_trts_real:.3f}  RMSE={rmse_trts_real:.3f}")
print(f"TRTS CopuOnly : R^2={r2_tstr_copu:.3f}  RMSE={rmse_tstr_copu:.3f}")


TSTR RealOnly : R^2=0.688  RMSE=1.849
TSTR CopuOnly : R^2=-0.079  RMSE=3.437
TRTS RealOnly : R^2=0.231  RMSE=1.589
TRTS CopuOnly : R^2=-0.079  RMSE=3.437


In [25]:
# Mix Data
tvae_1x = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu")

# 1 : 1
tvae_1x.fit(
    df_1x,
    epochs=300,
    warmup_epochs=30)

fake_1x = tvae_1x.sample(10_000)

# 1 : 2
tvae_2x = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu")

tvae_2x.fit(
    df_2x,
    epochs=300,
    warmup_epochs=30)

fake_2x = tvae_2x.sample(10_000)

# 1 : 4
tvae_4x = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu")

tvae_4x.fit(
    df_4x,
    epochs=300,
    warmup_epochs=30)

fake_4x = tvae_4x.sample(10_000)

# 1 : 8
tvae_8x = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu")

tvae_8x.fit(
    df_8x,
    epochs=300,
    warmup_epochs=30)

fake_8x = tvae_8x.sample(10_000)

Epoch 50/300 | Loss=0.4501 | MSE=0.2544 | KL=0.1957 | β=1.000
Epoch 100/300 | Loss=0.4017 | MSE=0.1950 | KL=0.2067 | β=1.000
Epoch 150/300 | Loss=0.3656 | MSE=0.1655 | KL=0.2001 | β=1.000
Epoch 200/300 | Loss=0.3321 | MSE=0.1362 | KL=0.1959 | β=1.000
Epoch 250/300 | Loss=0.3290 | MSE=0.1268 | KL=0.2022 | β=1.000
Epoch 300/300 | Loss=0.3211 | MSE=0.1138 | KL=0.2073 | β=1.000
Epoch 50/300 | Loss=0.4639 | MSE=0.2475 | KL=0.2164 | β=1.000
Epoch 100/300 | Loss=0.4289 | MSE=0.2124 | KL=0.2166 | β=1.000
Epoch 150/300 | Loss=0.3763 | MSE=0.1686 | KL=0.2077 | β=1.000
Epoch 200/300 | Loss=0.3406 | MSE=0.1282 | KL=0.2124 | β=1.000
Epoch 250/300 | Loss=0.3270 | MSE=0.1236 | KL=0.2034 | β=1.000
Epoch 300/300 | Loss=0.3422 | MSE=0.1223 | KL=0.2200 | β=1.000
Epoch 50/300 | Loss=0.5067 | MSE=0.2887 | KL=0.2180 | β=1.000
Epoch 100/300 | Loss=0.3978 | MSE=0.1645 | KL=0.2333 | β=1.000
Epoch 150/300 | Loss=0.3988 | MSE=0.1616 | KL=0.2372 | β=1.000
Epoch 200/300 | Loss=0.3263 | MSE=0.1037 | KL=0.2226 | β=1

In [26]:
# TSTR
r2_tstr_1x, rmse_tstr_1x = xgb_score(fake_1x, test_real)
r2_tstr_2x, rmse_tstr_2x = xgb_score(fake_2x, test_real)
r2_tstr_4x, rmse_tstr_4x = xgb_score(fake_4x, test_real)
r2_tstr_8x, rmse_tstr_8x = xgb_score(fake_8x, test_real)

# Print
print(f"TSTR 1x : R^2={r2_tstr_1x:.3f}  RMSE={rmse_tstr_1x:.3f}")
print(f"TSTR 2x : R^2={r2_tstr_2x:.3f}  RMSE={rmse_tstr_2x:.3f}")
print(f"TSTR 4x : R^2={r2_tstr_4x:.3f}  RMSE={rmse_tstr_4x:.3f}")
print(f"TSTR 8x : R^2={r2_tstr_8x:.3f}  RMSE={rmse_tstr_8x:.3f}")

TSTR 1x : R^2=0.524  RMSE=2.283
TSTR 2x : R^2=0.516  RMSE=2.302
TSTR 4x : R^2=0.285  RMSE=2.799
TSTR 8x : R^2=0.304  RMSE=2.760


In [27]:
# Only consider R^2 score for model validation
def downstream_r2(fake_df: pd.DataFrame,
                  real_val_df: pd.DataFrame,
                  target_col: str = None,
                  seed: int = 0) -> float:
    """

    • fake_df      : Synthetic data generated by TVAE.
    • real_val_df  : Data set for test.
    • target_col   : Target column name.
    """
    target_col = target_col or real_val_df.columns[-1]

    X_train = fake_df.drop(columns=[target_col]).values
    y_train = fake_df[target_col].values
    X_val   = real_val_df.drop(columns=[target_col]).values
    y_val   = real_val_df[target_col].values

    model = XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=seed,                  
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    return r2_score(y_val, preds)


In [28]:
# Test the distribution difference between synthetic data and real data

def _rbf_kernel(X, Y, gamma):
    """RBF kernel k(x,y) = exp(-gamma‖x-y‖²)"""
    dists = cdist(X, Y, 'sqeuclidean')
    return np.exp(-gamma * dists)

def compute_mmd(fake_df: pd.DataFrame,
                real_df: pd.DataFrame,
                gamma: float = None) -> float:
    """
    return Maximum Mean Discrepancy (MMD): The smaller value means more similar distribution.
    gamma is set to 1/(2 sigma^2) where sigma^2 is the mean characeristic variance of the merge of real & syn data
    """
    X = fake_df.values.astype(np.float64)
    Y = real_df.values.astype(np.float64)

    if gamma is None:
        var = np.var(np.vstack([X, Y]), axis=0).mean()
        gamma = 1.0 / (2 * var + 1e-8)

    Kxx = _rbf_kernel(X, X, gamma)
    Kyy = _rbf_kernel(Y, Y, gamma)
    Kxy = _rbf_kernel(X, Y, gamma)

    m = len(X)
    n = len(Y)

    mmd2 = (Kxx.sum() - np.trace(Kxx)) / (m * (m - 1)) \
         + (Kyy.sum() - np.trace(Kyy)) / (n * (n - 1)) \
         - 2 * Kxy.mean()

    return float(np.sqrt(max(mmd2, 0)))        


In [29]:
def objective(trial, df_train, meta):
    # Validation set
    ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=trial.number)
    tr_sub_idx, val_idx = next(ss.split(df_train))
    df_sub = df_train.iloc[tr_sub_idx].reset_index(drop=True)
    df_val = df_train.iloc[val_idx].reset_index(drop=True)

    # Hyperparameter Sampling
    h1 = trial.suggest_int('h1', 64, 256, step=64)
    h2 = trial.suggest_int('h2', 32, 128, step=32)
    latent = trial.suggest_int('latent', 16, 128, step=16)
    lr = trial.suggest_float('lr', 1e-4, 5e-3, log=True)   
    warm = trial.suggest_int('warmup', 5, 40)
    beta_max = trial.suggest_float('beta_max', 1.0, 4.0)
    sched = trial.suggest_categorical('sched', ['linear', 'sigmoid', 'cyclic'])

    # Build up models
    model = TVAEAnneal(
        metadata=meta,
        batch_size=256,
        device="cpu",
        latent_dim=latent,
        hidden_sizes=(h1, h2),
        beta_max=beta_max)

    model.fit(df_train,
              epochs=200,
              warmup_epochs=warm,
              lr=lr,
              sched=sched)

    # Evaluate and return metrics
    fake = model.sample(len(df_train))
    r2   = downstream_r2(fake, df_val)    
    mmd  = compute_mmd(fake, df_val)
    trial.set_user_attr("r2",  r2)
    trial.set_user_attr("mmd", mmd)
    score = r2 - 0.1 * mmd
    return score


In [30]:
study = optuna.create_study(
    study_name="anneal_tvae_search",
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42),  
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)  # Early-stop for obvious wrong trial
)

study.optimize(
    partial(objective,
            df_train=train_real,
            meta=meta),
    n_trials=50,          # Try 50 different combination
    timeout=60*60,        # Time limitation
    n_jobs=1,             
    show_progress_bar=True
)


[I 2025-08-18 00:58:28,732] A new study created in memory with name: anneal_tvae_search
  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 50/200 | Loss=0.3898 | MSE=0.2628 | KL=0.0865 | β=1.468
Epoch 100/200 | Loss=0.3228 | MSE=0.1831 | KL=0.0952 | β=1.468
Epoch 150/200 | Loss=0.2910 | MSE=0.1545 | KL=0.0930 | β=1.468
Epoch 200/200 | Loss=0.2783 | MSE=0.1388 | KL=0.0950 | β=1.468


Best trial: 0. Best value: 0.20783:   2%|▏         | 1/50 [00:00<00:26,  1.83it/s, 0.55/3600 seconds]

[I 2025-08-18 00:58:29,279] Trial 0 finished with value: 0.20783043233758203 and parameters: {'h1': 128, 'h2': 128, 'latent': 96, 'lr': 0.0010401663679887319, 'warmup': 10, 'beta_max': 1.4679835610086078, 'sched': 'sigmoid'}. Best is trial 0 with value: 0.20783043233758203.
Epoch 50/200 | Loss=0.2003 | MSE=0.2003 | KL=0.0749 | β=0.000
Epoch 100/200 | Loss=0.1618 | MSE=0.1618 | KL=0.0796 | β=0.000
Epoch 150/200 | Loss=0.1532 | MSE=0.1532 | KL=0.0782 | β=0.000
Epoch 200/200 | Loss=0.1225 | MSE=0.1225 | KL=0.0798 | β=0.000


Best trial: 1. Best value: 0.228629:   4%|▍         | 2/50 [00:01<00:26,  1.84it/s, 1.09/3600 seconds]

...

[I 2025-08-18 00:58:53,768] Trial 47 finished with value: 0.49798092607745115 and parameters: {'h1': 256, 'h2': 128, 'latent': 16, 'lr': 0.0022537473566420817, 'warmup': 25, 'beta_max': 3.6044813573892815, 'sched': 'cyclic'}. Best is trial 19 with value: 0.6369914046792744.
Epoch 50/200 | Loss=0.7179 | MSE=0.7179 | KL=0.0593 | β=0.000
Epoch 100/200 | Loss=0.5847 | MSE=0.5847 | KL=0.0744 | β=0.000
Epoch 150/200 | Loss=0.5226 | MSE=0.5226 | KL=0.0885 | β=0.000
Epoch 200/200 | Loss=0.4322 | MSE=0.4322 | KL=0.0912 | β=0.000


Best trial: 19. Best value: 0.636991:  98%|█████████▊| 49/50 [00:25<00:00,  1.97it/s, 25.52/3600 seconds]

[I 2025-08-18 00:58:54,251] Trial 48 finished with value: 0.3421152467360458 and parameters: {'h1': 128, 'h2': 96, 'latent': 16, 'lr': 0.001580250388004995, 'warmup': 22, 'beta_max': 3.420691246193001, 'sched': 'cyclic'}. Best is trial 19 with value: 0.6369914046792744.
Epoch 50/200 | Loss=0.4667 | MSE=0.4667 | KL=0.0624 | β=0.000
Epoch 100/200 | Loss=0.4333 | MSE=0.4333 | KL=0.0693 | β=0.000
Epoch 150/200 | Loss=0.4160 | MSE=0.4160 | KL=0.0714 | β=0.000
Epoch 200/200 | Loss=0.3573 | MSE=0.3573 | KL=0.0701 | β=0.000


Best trial: 19. Best value: 0.636991: 100%|██████████| 50/50 [00:25<00:00,  1.92it/s, 25.99/3600 seconds]

[I 2025-08-18 00:58:54,727] Trial 49 finished with value: 0.4370231801534821 and parameters: {'h1': 192, 'h2': 64, 'latent': 32, 'lr': 0.0025717083589782094, 'warmup': 33, 'beta_max': 3.9040303515974992, 'sched': 'cyclic'}. Best is trial 19 with value: 0.6369914046792744.


In [31]:
# 1) Overview Trial
best = study.best_trial
print("Best score:", best.value)
print("Params :", best.params)
print("R^2 =", best.user_attrs['r2'])

# 2) Results to DataFrame
df_trials = study.trials_dataframe(attrs=('number', 'value', 'params', 'user_attrs'))
df_trials.to_csv("TAVE_annealing_trials.csv", index=False)

Best score: 0.6369914046792744
Params : {'h1': 256, 'h2': 128, 'latent': 32, 'lr': 0.0033215769881425887, 'warmup': 34, 'beta_max': 2.620285060340782, 'sched': 'linear'}
R^2 = 0.6578663299118668


In [334]:
best_params = best.params
model_best = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (best_params['h1'], best_params['h2']),
    beta_max       = best_params['beta_max']
)

model_best.fit(
    train_real,
    epochs=300,
    warmup_epochs=best_params['warmup'],
    lr=best_params['lr'],
    sched=best_params['sched']
)

fake_best = model_best.sample(len(train_real))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing with real data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)


Epoch 50/300 | Loss=0.6195 | MSE=0.3516 | KL=0.1714 | β=1.563
Epoch 100/300 | Loss=0.5864 | MSE=0.3034 | KL=0.1811 | β=1.563
Epoch 150/300 | Loss=0.5270 | MSE=0.2463 | KL=0.1796 | β=1.563
Epoch 200/300 | Loss=0.5214 | MSE=0.2356 | KL=0.1829 | β=1.563
Epoch 250/300 | Loss=0.4931 | MSE=0.2127 | KL=0.1794 | β=1.563
Epoch 300/300 | Loss=0.5047 | MSE=0.2226 | KL=0.1805 | β=1.563
TSTR results of TVAE_annealing with real data input (R^2, RMSE): 0.5338771358006065 , 2.25905003475542
MMD: 0.17815335897210818


In [335]:
best_params = best.params
model_best = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (best_params['h1'], best_params['h2']),
    beta_max       = best_params['beta_max']
)

model_best.fit(
    train_copu,
    epochs=300,
    warmup_epochs=best_params['warmup'],
    lr=best_params['lr'],
    sched=best_params['sched']
)

fake_best = model_best.sample(len(train_copu))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing with Copula synthetic data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.7055 | MSE=0.3917 | KL=0.2008 | β=1.563
Epoch 100/300 | Loss=0.6761 | MSE=0.3601 | KL=0.2022 | β=1.563
Epoch 150/300 | Loss=0.6295 | MSE=0.3134 | KL=0.2022 | β=1.563
Epoch 200/300 | Loss=0.6678 | MSE=0.3422 | KL=0.2083 | β=1.563
Epoch 250/300 | Loss=0.7126 | MSE=0.3757 | KL=0.2156 | β=1.563
Epoch 300/300 | Loss=0.6763 | MSE=0.3551 | KL=0.2055 | β=1.563
TSTR results of TVAE_annealing with Copula synthetic data input (R^2, RMSE): -0.18233631132338068 , 3.59787574513955
MMD: 0.19360975761986343


In [336]:
best_params = best.params
model_best = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (best_params['h1'], best_params['h2']),
    beta_max       = best_params['beta_max']
)

model_best.fit(
    df_1x,
    epochs=300,
    warmup_epochs=best_params['warmup'],
    lr=best_params['lr'],
    sched=best_params['sched']
)

fake_best = model_best.sample(len(df_1x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing with 1:1 mixing data (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.7487 | MSE=0.4404 | KL=0.1973 | β=1.563
Epoch 100/300 | Loss=0.6041 | MSE=0.3183 | KL=0.1829 | β=1.563
Epoch 150/300 | Loss=0.6305 | MSE=0.3384 | KL=0.1869 | β=1.563
Epoch 200/300 | Loss=0.6653 | MSE=0.3599 | KL=0.1954 | β=1.563
Epoch 250/300 | Loss=0.6427 | MSE=0.3366 | KL=0.1958 | β=1.563
Epoch 300/300 | Loss=0.6577 | MSE=0.3390 | KL=0.2039 | β=1.563
TSTR results of TVAE_annealing with 1:1 mixing data (R^2, RMSE): 0.3936032191950072 , 2.5766420517909436
MMD: 0.19916061871709542


In [337]:
best_params = best.params
model_best = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (best_params['h1'], best_params['h2']),
    beta_max       = best_params['beta_max']
)

model_best.fit(
    df_2x,
    epochs=300,
    warmup_epochs=best_params['warmup'],
    lr=best_params['lr'],
    sched=best_params['sched']
)

fake_best = model_best.sample(len(df_2x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing with 1:2 mixing data (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.6886 | MSE=0.4091 | KL=0.1788 | β=1.563
Epoch 100/300 | Loss=0.6738 | MSE=0.3688 | KL=0.1952 | β=1.563
Epoch 150/300 | Loss=0.7393 | MSE=0.3898 | KL=0.2236 | β=1.563
Epoch 200/300 | Loss=0.6984 | MSE=0.3496 | KL=0.2231 | β=1.563
Epoch 250/300 | Loss=0.6325 | MSE=0.3226 | KL=0.1983 | β=1.563
Epoch 300/300 | Loss=0.5924 | MSE=0.3063 | KL=0.1830 | β=1.563
TSTR results of TVAE_annealing with 1:2 mixing data (R^2, RMSE): 0.5469189822966607 , 2.2272223727543934
MMD: 0.19951628045129266


In [338]:
best_params = best.params
model_best = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (best_params['h1'], best_params['h2']),
    beta_max       = best_params['beta_max']
)

model_best.fit(
    df_4x,
    epochs=300,
    warmup_epochs=best_params['warmup'],
    lr=best_params['lr'],
    sched=best_params['sched']
)

fake_best = model_best.sample(len(df_4x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing with 1:4 mixing data (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.7782 | MSE=0.4654 | KL=0.2001 | β=1.563
Epoch 100/300 | Loss=0.6295 | MSE=0.3692 | KL=0.1666 | β=1.563
Epoch 150/300 | Loss=0.6462 | MSE=0.3345 | KL=0.1994 | β=1.563
Epoch 200/300 | Loss=0.7032 | MSE=0.3899 | KL=0.2005 | β=1.563
Epoch 250/300 | Loss=0.7333 | MSE=0.4456 | KL=0.1841 | β=1.563
Epoch 300/300 | Loss=0.6011 | MSE=0.3588 | KL=0.1550 | β=1.563
TSTR results of TVAE_annealing with 1:4 mixing data (R^2, RMSE): 0.23747032056712192 , 2.889375745779571
MMD: 0.22796218212881475


In [340]:
best_params = best.params
model_best = TVAEAnneal(
    metadata=meta,
    batch_size=256,
    device="cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (best_params['h1'], best_params['h2']),
    beta_max       = best_params['beta_max']
)

model_best.fit(
    df_8x,
    epochs=300,
    warmup_epochs=best_params['warmup'],
    lr=best_params['lr'],
    sched=best_params['sched']
)

fake_best = model_best.sample(len(df_8x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing with 1:8 mixing data (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.7140 | MSE=0.4130 | KL=0.1926 | β=1.563
Epoch 100/300 | Loss=0.6977 | MSE=0.3863 | KL=0.1992 | β=1.563
Epoch 150/300 | Loss=0.7024 | MSE=0.3880 | KL=0.2012 | β=1.563
Epoch 200/300 | Loss=0.6108 | MSE=0.3108 | KL=0.1920 | β=1.563
Epoch 250/300 | Loss=0.6573 | MSE=0.3474 | KL=0.1983 | β=1.563
Epoch 300/300 | Loss=0.7004 | MSE=0.3679 | KL=0.2127 | β=1.563
TSTR results of TVAE_annealing with 1:8 mixing data (R^2, RMSE): 0.17223897014135148 , 3.0104272397656606
MMD: 0.23099350770856247


In [341]:

# Define for training and saving hyperparameters
def train_and_save(name, latent_dim, beta_max,
                   epochs=300, warmup=32, lr=0.0017,
                   hidden_sizes=(256, 96), device="cpu"):

    model = TVAEAnneal(
        metadata     = meta,
        latent_dim   = latent_dim,
        hidden_sizes = hidden_sizes,
        beta_max     = beta_max,
        batch_size   = 128,
        device       = device)

    model.fit(
        train_real,
        epochs        = epochs,
        warmup_epochs = warmup,
        lr            = lr,
        sched         = "sigmoid" )

    # Save hyperparameters
    weight_path = f"tvae_{name}.pt"
    torch.save(model.model.state_dict(), weight_path)
    print(f"Saved weights: {weight_path}")

    # Generate synthetic data
    fake = model.sample(10_000)
    csv_path = f"Synthetic_{name}.csv"
    fake.to_csv(csv_path, index=False)
    print(f"Saved samples: {csv_path}")

# Best Result
train_and_save(
    name       = "TAVE_annealing",  
    latent_dim  = 32,
    beta_max    = 2.0229147092990662,
    lr          = 0.004568164246533539,
    warmup      = 40,
    hidden_sizes= (256, 128),    
    device      = "cpu")       


Epoch 50/300 | Loss=0.5183 | MSE=0.2824 | KL=0.1179 | β=2.001
Epoch 100/300 | Loss=0.4666 | MSE=0.2114 | KL=0.1262 | β=2.023
Epoch 150/300 | Loss=0.4487 | MSE=0.2112 | KL=0.1174 | β=2.023
Epoch 200/300 | Loss=0.3852 | MSE=0.1635 | KL=0.1096 | β=2.023
Epoch 250/300 | Loss=0.3625 | MSE=0.1269 | KL=0.1164 | β=2.023
Epoch 300/300 | Loss=0.3941 | MSE=0.1354 | KL=0.1279 | β=2.023
Saved weights: tvae_TAVE_annealing.pt
Saved samples: Synthetic_TAVE_annealing.csv


In [342]:
class TVAEAnnealGMM(TVAEAnneal):
    """
    Inherit from TVAE-annealing and adding extra Bayesian GMM to latent space
    sample() with TVAE-GMM by default
    """
    def __init__(self, *args,
                 gmm_components=50,     # Max combination number
                 gmm_cov='diag',        # 'full' or 'diag'
                 **kws):
        super().__init__(*args, **kws)
        self.gmm_components = gmm_components
        self.gmm_cov = gmm_cov
        self.gmm = None                

    # fit function
    def fit(self, df: pd.DataFrame,
            epochs=300, warmup_epochs=30, lr=2e-3,
            sched='linear'):
        # Regular TVAE 
        super().fit(df, epochs, warmup_epochs, lr, sched)

        # Collect latent vector muon as GMM training data
        tensor_data = self.scaler.transform(df)
        dl = DataLoader(TensorDataset(tensor_data),
                        batch_size=1024, shuffle=False)
        self.model.eval()
        mu_list = []
        with torch.no_grad():
            for (x,) in dl:
                x = x.to(self.device)
                mu, _ = self.model.encode(x)
                mu_list.append(mu.cpu())
        Z = torch.cat(mu_list, dim=0).numpy()

        # Simulate Bayesian GMM
        self.gmm = BayesianGaussianMixture(
            n_components = self.gmm_components,
            covariance_type = self.gmm_cov,
            weight_concentration_prior_type="dirichlet_process",
            max_iter=500,
            random_state=0
        ).fit(Z)

        print(f"BGM fitted: active comps = "
              f"{np.sum(self.gmm.weights_ > 1e-3)}/{self.gmm_components}")

    # sample function
    def sample(self, n: int, use_gmm=True) -> pd.DataFrame:
        self.model.eval()
        with torch.no_grad():
            if use_gmm and self.gmm is not None:
                z_np = self.gmm.sample(n)[0].astype(np.float32)
                z = torch.from_numpy(z_np).to(self.device)
            else:
                z = torch.randn(n, self.latent).to(self.device)

            gen = self.model.decode(z)
        return self.scaler.inverse_transform(gen)

    # Save and load
    def save(self, path_prefix: str):
        torch.save(self.model.state_dict(), f"{path_prefix}_vae.pt")
        if self.gmm is not None:
            joblib.dump(self.gmm, f"{path_prefix}_gmm.pkl")
        self.scaler.scaler.mean_.tofile(f"{path_prefix}_mean.bin")
        self.scaler.scaler.scale_.tofile(f"{path_prefix}_scale.bin")

    def load(self, path_prefix: str):
        self.model.load_state_dict(torch.load(f"{path_prefix}_vae.pt"))
        self.gmm = joblib.load(f"{path_prefix}_gmm.pkl")
        

In [343]:
# Training
tvae_gmm = TVAEAnnealGMM(
    metadata      = meta,
    latent_dim    = 32,
    hidden_sizes  = (256, 128),
    beta_max      = 1.80,
    batch_size    = 128,
    device        = "cpu",          
    gmm_components= 50,              # Important hyperparameter for GMM
    gmm_cov       ='diag')

tvae_gmm.fit(train_real,
             epochs=300,
             warmup_epochs=32,
             lr=0.0040,
             sched='sigmoid')

# Sampling
fake_gmm = tvae_gmm.sample(10_000)          

Epoch 50/300 | Loss=0.5095 | MSE=0.2752 | KL=0.1304 | β=1.797
Epoch 100/300 | Loss=0.4805 | MSE=0.2538 | KL=0.1259 | β=1.800
Epoch 150/300 | Loss=0.3973 | MSE=0.1930 | KL=0.1136 | β=1.800
Epoch 200/300 | Loss=0.3642 | MSE=0.1416 | KL=0.1236 | β=1.800
Epoch 250/300 | Loss=0.3430 | MSE=0.1364 | KL=0.1148 | β=1.800
Epoch 300/300 | Loss=0.3569 | MSE=0.1445 | KL=0.1180 | β=1.800
BGM fitted: active comps = 50/50


In [344]:
target_col = "fragmentation_index" 
r2  = downstream_r2(fake_gmm,  test_real, target_col)
mmd = compute_mmd  (fake_gmm.drop(columns=[target_col]),
                    test_real.drop(columns=[target_col]))
print(f"GMM-TVAE: R^2={r2:.3f}, MMD={mmd:.3f}")


GMM-TVAE: R^2=0.681, MMD=0.101


In [ ]:
def gmm_objective(trial, df_train, meta):
    # Validation set
    ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=trial.number)
    tr_sub_idx, val_idx = next(ss.split(df_train))
    df_sub = df_train.iloc[tr_sub_idx].reset_index(drop=True)  
    df_val = df_train.iloc[val_idx].reset_index(drop=True) 

    latent = trial.suggest_categorical("latent", [24,28,32,36,40])
    beta   = trial.suggest_float("beta", 1.4, 2.2)
    K      = trial.suggest_categorical("gmm_K", [12,20,28,36])
    lr     = trial.suggest_float("lr", 3e-3, 6e-3, log=True)
    warm   = trial.suggest_categorical("warm", [16,20,24,28])

    model = TVAEAnnealGMM(
        metadata=meta,
        latent_dim=latent,
        hidden_sizes=(256,128),
        beta_max=beta,
        gmm_components=K,
        gmm_cov='diag',
        batch_size=128,
        device='cpu')

    model.fit(df_sub,
              epochs=250,
              warmup_epochs=warm,
              lr=lr,
              sched='sigmoid')

    fake = model.sample(len(df_train))
    r2   = downstream_r2(fake, df_val)
    mmd  = compute_mmd(fake, df_val)

    score = r2 - 0.1*mmd            # Similar metrics combining r^2 and mmd
    trial.set_user_attr("r2", r2)
    trial.set_user_attr("mmd", mmd)
    return score

study = optuna.create_study(direction="maximize")
study.optimize(lambda t: gmm_objective(t, train_real, meta),
               n_trials=25, timeout=15*60, show_progress_bar=True)
print("Best params:", study.best_trial.params,
      "R^2 =", study.best_trial.user_attrs['r2'],
      "MMD =", study.best_trial.user_attrs['mmd'])


[I 2025-08-06 23:59:42,217] A new study created in memory with name: no-name-1ba54a31-276d-4e28-8186-ff0b83316e12
  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 50/250 | Loss=0.4443 | MSE=0.2345 | KL=0.1391 | β=1.508
Epoch 100/250 | Loss=0.3574 | MSE=0.1688 | KL=0.1250 | β=1.508
Epoch 150/250 | Loss=0.2875 | MSE=0.1075 | KL=0.1193 | β=1.508
Epoch 200/250 | Loss=0.2707 | MSE=0.1095 | KL=0.1069 | β=1.508


Best trial: 0. Best value: 0.505622:   4%|▍         | 1/25 [00:00<00:19,  1.20it/s, 0.83/900 seconds]

Epoch 250/250 | Loss=0.2625 | MSE=0.1034 | KL=0.1054 | β=1.508
BGM fitted: active comps = 28/28
[I 2025-08-06 23:59:43,048] Trial 0 finished with value: 0.5056215843593223 and parameters: {'latent': 40, 'beta': 1.5082506317057154, 'gmm_K': 28, 'lr': 0.0032490854875309, 'warm': 20}. Best is trial 0 with value: 0.5056215843593223.
Epoch 50/250 | Loss=0.5765 | MSE=0.3204 | KL=0.1262 | β=2.030
Epoch 100/250 | Loss=0.4899 | MSE=0.2293 | KL=0.1284 | β=2.030
Epoch 150/250 | Loss=0.4067 | MSE=0.1683 | KL=0.1174 | β=2.030
Epoch 200/250 | Loss=0.3894 | MSE=0.1579 | KL=0.1140 | β=2.030


Best trial: 1. Best value: 0.563253:   8%|▊         | 2/25 [00:01<00:19,  1.20it/s, 1.67/900 seconds]

...

[I 2025-08-07 00:00:01,609] Trial 22 finished with value: 0.6384157089640426 and parameters: {'latent': 40, 'beta': 2.101331835117563, 'gmm_K': 28, 'lr': 0.005994241580606686, 'warm': 20}. Best is trial 17 with value: 0.7095161834382396.
Epoch 50/250 | Loss=0.4443 | MSE=0.2562 | KL=0.1017 | β=1.850
Epoch 100/250 | Loss=0.3878 | MSE=0.1858 | KL=0.1091 | β=1.850
Epoch 150/250 | Loss=0.3460 | MSE=0.1598 | KL=0.1006 | β=1.850
Epoch 200/250 | Loss=0.3204 | MSE=0.1357 | KL=0.0998 | β=1.850
Epoch 250/250 | Loss=0.3135 | MSE=0.1175 | KL=0.1060 | β=1.850
BGM fitted: active comps = 28/28


Best trial: 17. Best value: 0.709516:  96%|█████████▌| 24/25 [00:20<00:00,  1.17it/s, 20.22/900 seconds]

[I 2025-08-07 00:00:02,442] Trial 23 finished with value: 0.41406744200828943 and parameters: {'latent': 40, 'beta': 1.8501943093573596, 'gmm_K': 28, 'lr': 0.005254065370991711, 'warm': 20}. Best is trial 17 with value: 0.7095161834382396.
Epoch 50/250 | Loss=0.4692 | MSE=0.2477 | KL=0.1106 | β=2.002
Epoch 100/250 | Loss=0.4026 | MSE=0.2090 | KL=0.0966 | β=2.003
Epoch 150/250 | Loss=0.3699 | MSE=0.1318 | KL=0.1189 | β=2.003
Epoch 200/250 | Loss=0.3715 | MSE=0.1332 | KL=0.1190 | β=2.003
Epoch 250/250 | Loss=0.3345 | MSE=0.1270 | KL=0.1036 | β=2.003
BGM fitted: active comps = 28/28


Best trial: 17. Best value: 0.709516: 100%|██████████| 25/25 [00:21<00:00,  1.19it/s, 21.03/900 seconds]

[I 2025-08-07 00:00:03,245] Trial 24 finished with value: 0.38235469437716185 and parameters: {'latent': 36, 'beta': 2.002814562968468, 'gmm_K': 28, 'lr': 0.004852601129281566, 'warm': 28}. Best is trial 17 with value: 0.7095161834382396.
Best params: {'latent': 24, 'beta': 2.0352837502306653, 'gmm_K': 28, 'lr': 0.0059558380505119754, 'warm': 28} R^2 = 0.7180610162478024 MMD = 0.0854483280956274


In [349]:
best = study.best_trial
print("Best score :", f"{best.value:.5f}")
print("Best params:", best.params)
print("Best R^2    :", f"{best.user_attrs['r2']:.4f}")
print("Best MMD   :", f"{best.user_attrs['mmd']:.4f}")

df_trials = study.trials_dataframe(attrs=("number", "state", "value", "params", "user_attrs"))
df_trials.to_csv("TVAE_anneal_gmm_trials.csv", index=False)

Best score : 0.70952
Best params: {'latent': 24, 'beta': 2.0352837502306653, 'gmm_K': 28, 'lr': 0.0059558380505119754, 'warm': 28}
Best R^2    : 0.7181
Best MMD   : 0.0854


In [351]:
best_params = best.params
model_best = TVAEAnnealGMM(
    metadata       = meta,
    batch_size     = 256,
    device         = "cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (256, 128),             
    beta_max       = best_params['beta'],
    gmm_components = best_params['gmm_K'],
    gmm_cov        = 'diag'                  
)

model_best.fit(
    train_real,
    epochs        = 300,
    warmup_epochs = best_params['warm'],
    lr            = best_params['lr'],
    sched         = 'sigmoid'                
)

fake_best = model_best.sample(len(train_real))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing_GMM with real data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.6149 | MSE=0.3676 | KL=0.1215 | β=2.034
Epoch 100/300 | Loss=0.5268 | MSE=0.2562 | KL=0.1329 | β=2.035
Epoch 150/300 | Loss=0.4796 | MSE=0.2323 | KL=0.1215 | β=2.035
Epoch 200/300 | Loss=0.4479 | MSE=0.1682 | KL=0.1374 | β=2.035
Epoch 250/300 | Loss=0.4393 | MSE=0.1931 | KL=0.1210 | β=2.035
Epoch 300/300 | Loss=0.4300 | MSE=0.1705 | KL=0.1275 | β=2.035
BGM fitted: active comps = 28/28
TSTR results of TVAE_annealing_GMM with real data input (R^2, RMSE): 0.6353864838325292 , 1.9979841044735147
MMD: 0.10509308064636685


In [352]:
best_params = best.params
model_best = TVAEAnnealGMM(
    metadata       = meta,
    batch_size     = 256,
    device         = "cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (256, 128),             
    beta_max       = best_params['beta'],
    gmm_components = best_params['gmm_K'],
    gmm_cov        = 'diag'                  
)

model_best.fit(
    train_copu,
    epochs        = 300,
    warmup_epochs = best_params['warm'],
    lr            = best_params['lr'],
    sched         = 'sigmoid'                
)

fake_best = model_best.sample(len(train_copu))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing_GMM with Copula data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.6516 | MSE=0.3373 | KL=0.1545 | β=2.034
Epoch 100/300 | Loss=0.6428 | MSE=0.3126 | KL=0.1622 | β=2.035
Epoch 150/300 | Loss=0.6338 | MSE=0.3158 | KL=0.1562 | β=2.035
Epoch 200/300 | Loss=0.6523 | MSE=0.3287 | KL=0.1590 | β=2.035
Epoch 250/300 | Loss=0.6365 | MSE=0.3163 | KL=0.1573 | β=2.035
Epoch 300/300 | Loss=0.6428 | MSE=0.3171 | KL=0.1600 | β=2.035
BGM fitted: active comps = 27/28
TSTR results of TVAE_annealing_GMM with Copula data input (R^2, RMSE): -0.3078293224555666 , 3.784000472766599
MMD: 0.24768935276694826


In [353]:
best_params = best.params
model_best = TVAEAnnealGMM(
    metadata       = meta,
    batch_size     = 256,
    device         = "cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (256, 128),             
    beta_max       = best_params['beta'],
    gmm_components = best_params['gmm_K'],
    gmm_cov        = 'diag'                  
)

model_best.fit(
    df_1x,
    epochs        = 300,
    warmup_epochs = best_params['warm'],
    lr            = best_params['lr'],
    sched         = 'sigmoid'                
)

fake_best = model_best.sample(len(df_1x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing_GMM with 1:1 mixing data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.6872 | MSE=0.4078 | KL=0.1373 | β=2.034
Epoch 100/300 | Loss=0.6461 | MSE=0.3477 | KL=0.1466 | β=2.035
Epoch 150/300 | Loss=0.5867 | MSE=0.2805 | KL=0.1504 | β=2.035
Epoch 200/300 | Loss=0.6221 | MSE=0.2969 | KL=0.1598 | β=2.035
Epoch 250/300 | Loss=0.5836 | MSE=0.2827 | KL=0.1479 | β=2.035
Epoch 300/300 | Loss=0.5837 | MSE=0.2769 | KL=0.1507 | β=2.035
BGM fitted: active comps = 28/28
TSTR results of TVAE_annealing_GMM with 1:1 mixing data input (R^2, RMSE): 0.5970392688251108 , 2.1004241826269796
MMD: 0.1589433402744021


In [354]:
best_params = best.params
model_best = TVAEAnnealGMM(
    metadata       = meta,
    batch_size     = 256,
    device         = "cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (256, 128),             
    beta_max       = best_params['beta'],
    gmm_components = best_params['gmm_K'],
    gmm_cov        = 'diag'                  
)

model_best.fit(
    df_2x,
    epochs        = 300,
    warmup_epochs = best_params['warm'],
    lr            = best_params['lr'],
    sched         = 'sigmoid'                
)

fake_best = model_best.sample(len(df_2x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing_GMM with 1:2 mixing data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.6514 | MSE=0.3584 | KL=0.1440 | β=2.034
Epoch 100/300 | Loss=0.6456 | MSE=0.3410 | KL=0.1497 | β=2.035
Epoch 150/300 | Loss=0.6523 | MSE=0.3122 | KL=0.1671 | β=2.035
Epoch 200/300 | Loss=0.5670 | MSE=0.2658 | KL=0.1480 | β=2.035
Epoch 250/300 | Loss=0.5846 | MSE=0.2895 | KL=0.1450 | β=2.035
Epoch 300/300 | Loss=0.5755 | MSE=0.2743 | KL=0.1480 | β=2.035
BGM fitted: active comps = 28/28
TSTR results of TVAE_annealing_GMM with 1:2 mixing data input (R^2, RMSE): 0.4454384671026135 , 2.464055681512016
MMD: 0.1850200714138146


In [355]:
best_params = best.params
model_best = TVAEAnnealGMM(
    metadata       = meta,
    batch_size     = 256,
    device         = "cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (256, 128),             
    beta_max       = best_params['beta'],
    gmm_components = best_params['gmm_K'],
    gmm_cov        = 'diag'                  
)

model_best.fit(
    df_4x,
    epochs        = 300,
    warmup_epochs = best_params['warm'],
    lr            = best_params['lr'],
    sched         = 'sigmoid'                
)

fake_best = model_best.sample(len(df_4x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing_GMM with 1:4 mixing data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.6785 | MSE=0.3831 | KL=0.1452 | β=2.034
Epoch 100/300 | Loss=0.5495 | MSE=0.2964 | KL=0.1244 | β=2.035
Epoch 150/300 | Loss=0.6835 | MSE=0.3691 | KL=0.1545 | β=2.035
Epoch 200/300 | Loss=0.5797 | MSE=0.2596 | KL=0.1573 | β=2.035
Epoch 250/300 | Loss=0.6440 | MSE=0.3805 | KL=0.1295 | β=2.035
Epoch 300/300 | Loss=0.7075 | MSE=0.3537 | KL=0.1738 | β=2.035
BGM fitted: active comps = 28/28
TSTR results of TVAE_annealing_GMM with 1:4 mixing data input (R^2, RMSE): 0.41068748320061965 , 2.5400863434894134
MMD: 0.2762530813192122


In [356]:
best_params = best.params
model_best = TVAEAnnealGMM(
    metadata       = meta,
    batch_size     = 256,
    device         = "cpu",
    latent_dim     = best_params['latent'],
    hidden_sizes   = (256, 128),             
    beta_max       = best_params['beta'],
    gmm_components = best_params['gmm_K'],
    gmm_cov        = 'diag'                  
)

model_best.fit(
    df_8x,
    epochs        = 300,
    warmup_epochs = best_params['warm'],
    lr            = best_params['lr'],
    sched         = 'sigmoid'                
)

fake_best = model_best.sample(len(df_8x))

# Check R^2 score and RMSE
r2_best, rmse_best = xgb_score(fake_best, test_real)
print("TSTR results of TVAE_annealing_GMM with 1:8 mixing data input (R^2, RMSE):", r2_best, ",", rmse_best)

# Check the distribution discrepancy
mmd_best = compute_mmd(fake_best, test_real)
print("MMD:", mmd_best)

Epoch 50/300 | Loss=0.6806 | MSE=0.3180 | KL=0.1783 | β=2.034
Epoch 100/300 | Loss=0.6842 | MSE=0.3501 | KL=0.1642 | β=2.035
Epoch 150/300 | Loss=0.6347 | MSE=0.3226 | KL=0.1533 | β=2.035
Epoch 200/300 | Loss=0.6184 | MSE=0.3082 | KL=0.1525 | β=2.035
Epoch 250/300 | Loss=0.6010 | MSE=0.2863 | KL=0.1546 | β=2.035
Epoch 300/300 | Loss=0.6415 | MSE=0.3429 | KL=0.1467 | β=2.035
BGM fitted: active comps = 28/28
TSTR results of TVAE_annealing_GMM with 1:8 mixing data input (R^2, RMSE): 0.3409601273776601 , 2.6861576155860787
MMD: 0.2588595462283399


In [357]:
def train_and_save_gmm(name,
                       latent_dim,
                       beta_max,
                       gmm_components,
                       gmm_cov        ='diag',
                       epochs         =300,
                       warmup         =32,
                       lr             =0.0035,
                       hidden_sizes   =(256, 128),
                       batch_size     =128,
                       device         ="cpu"):

    model = TVAEAnnealGMM(
        metadata       = meta,
        latent_dim     = latent_dim,
        hidden_sizes   = hidden_sizes,
        beta_max       = beta_max,
        batch_size     = batch_size,
        device         = device,
        gmm_components = gmm_components,
        gmm_cov        = gmm_cov
    )

    model.fit(train_real,
              epochs        = epochs,
              warmup_epochs = warmup,
              lr            = lr,
              sched         = "sigmoid")

    # Save hyperparameters weights
    weight_path = f"tvae_gmm_{name}.pt"
    torch.save(model.model.state_dict(), weight_path)
    print(f"Saved weights: {weight_path}")

    # Save synthetic data
    fake = model.sample(10_000)        
    csv_path = f"Synthetic_TVAE_gmm_{name}.csv"
    fake.to_csv(csv_path, index=False)
    print(f"Saved samples to {csv_path}")


# Best Results
train_and_save_gmm(
    name            = "best",
    latent_dim      = 36,
    beta_max        = 1.45178048,
    gmm_components  = 36,
    lr              = 0.0054798077,
    warmup          = 20
)


Epoch 50/300 | Loss=0.4290 | MSE=0.2129 | KL=0.1489 | β=1.452
Epoch 100/300 | Loss=0.3444 | MSE=0.1632 | KL=0.1248 | β=1.452
Epoch 150/300 | Loss=0.2920 | MSE=0.1039 | KL=0.1295 | β=1.452
Epoch 200/300 | Loss=0.2925 | MSE=0.1163 | KL=0.1214 | β=1.452
Epoch 250/300 | Loss=0.2674 | MSE=0.0849 | KL=0.1257 | β=1.452
Epoch 300/300 | Loss=0.2837 | MSE=0.1035 | KL=0.1241 | β=1.452
BGM fitted: active comps = 36/36
Saved weights: tvae_gmm_best.pt
Saved samples to Synthetic_TVAE_gmm_best.csv


To check the robustness of TVAE-annealing and TVAE-GMM, we need to do more tests.

In [358]:
N_SAMPLE   = 10_000     # Sampling
N_SEEDS    = 10         # Try 10 different seed
BATCH_EVAL = 256

# Load in data
train_df = train_real
test_df  = test_real

In [359]:
# Best Hypereparameters of searching
cfg_anneal = dict(latent_dim=32, hidden_sizes=(256,128),
                  beta_max=2.023, 
                  batch_size=BATCH_EVAL)

cfg_gmm    = dict(latent_dim=36, hidden_sizes=(256,128),
                  beta_max=1.452, gmm_components=36,
                  batch_size=BATCH_EVAL)

In [360]:
# Build up models
def load_model(path, cfg, train_data,is_gmm=False):
    # instantiate TVAE models
    if is_gmm:
        m = TVAEAnnealGMM(metadata=meta,
                          latent_dim     = cfg['latent_dim'],
                          hidden_sizes   = cfg['hidden_sizes'],
                          beta_max       = cfg['beta_max'],
                          gmm_components = cfg['gmm_components'],
                          batch_size     = cfg['batch_size'],
                          device         ="cpu")
    else:
        m = TVAEAnneal(metadata=meta,
                       latent_dim   = cfg['latent_dim'],
                       hidden_sizes = cfg['hidden_sizes'],
                       beta_max     = cfg['beta_max'],
                       batch_size   = cfg['batch_size'],
                       device       ="cpu")

    # Create same VAE model 
    input_dim = len(train_data.columns)
    m.model = VAE(input_dim=input_dim,
                  latent_dim=cfg['latent_dim'],
                  h=list(cfg['hidden_sizes'])).to("cpu")
    state = torch.load(path, map_location="cpu")
    m.model.load_state_dict(state)
    m.model.eval()
    m.scaler.fit(train_data) 

    return m


In [361]:
# Evaluation
def eval_once(model, seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    fake = model.sample(N_SAMPLE)
    r2, rmse = xgb_score(fake, test_df)
    mmd      = compute_mmd(fake, test_df)
    return dict(r2=r2, rmse=rmse, mmd=mmd)

def robustness_summary(path, cfg, train_data, is_gmm=False):
    model = load_model(path, cfg, train_data, is_gmm)
    stats = [eval_once(model, s) for s in range(N_SEEDS)]
    df    = pd.DataFrame(stats)
    return df.agg(['mean', 'std']).T      

ann_df_real = robustness_summary("tvae_annealing_best.pt", cfg_anneal, train_df, is_gmm=False)
gmm_df_real = robustness_summary("tvae_gmm_best.pt",       cfg_gmm   , train_df, is_gmm=True)

summary = pd.concat({'Anneal_real': ann_df_real, 'GMM_real': gmm_df_real}, axis=1)
print(summary)

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_69838/3974650999.py:25: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000

     Anneal_real            GMM_real          
            mean       std      mean       std
r2      0.646359  0.016250  0.736284  0.018010
rmse    1.967217  0.045515  1.698295  0.058361
mmd     0.164021  0.002898  0.160267  0.002881


In [362]:
ann_df_1x = robustness_summary("tvae_annealing_best.pt", cfg_anneal, df_1x, is_gmm=False)
gmm_df_1x = robustness_summary("tvae_gmm_best.pt",       cfg_gmm   , df_1x, is_gmm=True)

summary = pd.concat({'Anneal_1x': ann_df_real, 'GMM_1x': gmm_df_real}, axis=1)
print(summary)

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_69838/3974650999.py:25: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000

     Anneal_1x              GMM_1x          
          mean       std      mean       std
r2    0.646359  0.016250  0.736284  0.018010
rmse  1.967217  0.045515  1.698295  0.058361
mmd   0.164021  0.002898  0.160267  0.002881


In [363]:
ann_df_2x = robustness_summary("tvae_annealing_best.pt", cfg_anneal, df_2x, is_gmm=False)
gmm_df_2x = robustness_summary("tvae_gmm_best.pt",       cfg_gmm   , df_2x, is_gmm=True)

summary = pd.concat({'Anneal_2x': ann_df_2x, 'GMM_2x': gmm_df_2x}, axis=1)
print(summary)

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_69838/3974650999.py:25: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000

     Anneal_2x              GMM_2x          
          mean       std      mean       std
r2    0.634801  0.012507  0.731600  0.016150
rmse  1.999322  0.034446  1.713526  0.051494
mmd   0.174965  0.002582  0.175812  0.002877


In [364]:
ann_df_4x = robustness_summary("tvae_annealing_best.pt", cfg_anneal, df_4x, is_gmm=False)
gmm_df_4x = robustness_summary("tvae_gmm_best.pt",       cfg_gmm   , df_4x, is_gmm=True)

summary = pd.concat({'Anneal_4x': ann_df_4x, 'GMM_4x': gmm_df_4x}, axis=1)
print(summary)

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_69838/3974650999.py:25: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000

     Anneal_4x              GMM_4x          
          mean       std      mean       std
r2    0.630149  0.013018  0.728528  0.018949
rmse  2.011995  0.035786  1.723047  0.060526
mmd   0.173634  0.002665  0.173954  0.002829


In [365]:
ann_df_8x = robustness_summary("tvae_annealing_best.pt", cfg_anneal, df_8x, is_gmm=False)
gmm_df_8x = robustness_summary("tvae_gmm_best.pt",       cfg_gmm   , df_8x, is_gmm=True)

summary = pd.concat({'Anneal_8x': ann_df_8x, 'GMM_8x': gmm_df_8x}, axis=1)
print(summary)

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_69838/3974650999.py:25: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000

     Anneal_8x              GMM_8x          
          mean       std      mean       std
r2    0.636911  0.020213  0.727856  0.017429
rmse  1.993093  0.056013  1.725319  0.055933
mmd   0.170316  0.002850  0.168751  0.002826


In [366]:
ann_copu = robustness_summary("tvae_annealing_best.pt", cfg_anneal, train_copu, is_gmm=False)
gmm_copu = robustness_summary("tvae_gmm_best.pt",       cfg_gmm   , train_copu, is_gmm=True)

summary = pd.concat({'Anneal_copu': ann_copu, 'GMM_copu': gmm_copu}, axis=1)
print(summary)

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_69838/3974650999.py:25: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000

     Anneal_copu            GMM_copu          
            mean       std      mean       std
r2      0.636461  0.012168  0.728746  0.019223
rmse    1.994787  0.033370  1.722324  0.061486
mmd     0.166216  0.002877  0.163160  0.002897


In [374]:
# Save data generated by TAVE_gmm + Copula
gmm_copu = load_model("tvae_gmm_best.pt", cfg_gmm, train_copu, is_gmm=True)
fake_gmm_copu  = gmm_copu.sample(10_000)
fake_gmm_copu.to_csv("Synthetic_TAVE_gmm_Copula.csv", index=False)
print("Saved 10k GMM-synthetic rows to Synthetic_TAVE_gmm_Copula.csv")

Saved 10k GMM-synthetic rows to Synthetic_TAVE_gmm_Copula.csv


/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_69838/3974650999.py:25: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Basing on the paper "Evaluating Generative Models for Tabular Data: Novel Metrics and Benchmarking", we can apply `FAED` to check the quality of synthetic data. 

In [375]:
df_gen = pd.read_csv("Synthetic_TAVE_gmm_Copula.csv")
df_gen = df_gen[train_real.columns]

df_gen_real = pd.read_csv("Synthetic_TVAE_gmm_best.csv")
df_gen_real = df_gen_real[train_real.columns]

In [376]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_real_scaled = scaler.fit_transform(train_real)
X_gen_scaled = scaler.transform(df_gen)
X_gen_real_scaled =  scaler.transform(df_gen_real)

In [377]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim=16):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat, z

X_real_tensor = torch.tensor(X_real_scaled, dtype=torch.float32)
loader = DataLoader(TensorDataset(X_real_tensor), batch_size=64, shuffle=True)

model = AutoEncoder(input_dim=X_real_tensor.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

model.train()
for epoch in range(100):
    for batch in loader:
        x = batch[0]
        x_hat, _ = model(x)
        loss = criterion(x_hat, x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [378]:
model.eval()
with torch.no_grad():
    z_real = model.encoder(torch.tensor(X_real_scaled, dtype=torch.float32))
    z_gen = model.encoder(torch.tensor(X_gen_scaled, dtype=torch.float32))
    z_gen_real = model.encoder(torch.tensor(X_gen_real_scaled, dtype=torch.float32))

In [379]:
import scipy.linalg

def compute_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    covmean, _ = scipy.linalg.sqrtm(sigma1 @ sigma2, disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return np.sum((mu1 - mu2) ** 2) + np.trace(sigma1 + sigma2 - 2 * covmean)

z_real_np = z_real.numpy()
z_gen_np = z_gen.numpy()
z_gen_real_np = z_gen_real.numpy()

mu_r, sigma_r = np.mean(z_real_np, axis=0), np.cov(z_real_np, rowvar=False)
mu_g, sigma_g = np.mean(z_gen_np, axis=0), np.cov(z_gen_np, rowvar=False)
mu_gr, sigma_gr = np.mean(z_gen_real_np, axis=0), np.cov(z_gen_real_np, rowvar=False)

copu_faed_score = compute_frechet_distance(mu_r, sigma_r, mu_g, sigma_g)
real_faed_score = compute_frechet_distance(mu_r, sigma_r, mu_gr, sigma_gr)
print("Copu_FAED score =", copu_faed_score)
print("real_FAED score =", real_faed_score)

Copu_FAED score = 1.6605847251875836
real_FAED score = 0.43598867447289397
